![](img/572_banner.png)

# Lab 2: Stochastic Gradient Descent & Pytorch

**Tomas Beuzen, January 2021**

![](img/torch.png)

## Table of Contents
<hr>

<div class="toc"><ul class="toc-item"><li><span><a href="#Instructions" data-toc-modified-id="Instructions-2">Instructions</a></span></li><li><span><a href="#Imports" data-toc-modified-id="Imports-3">Imports</a></span></li><li><span><a href="#Exercise-1:-Stochastic-Gradient-Descent" data-toc-modified-id="Exercise-1:-Stochastic-Gradient-Descent-4">Exercise 1: Stochastic Gradient Descent</a></span></li><li><span><a href="#Exercise-2:-SGDClassifier-and-SGDRegresor" data-toc-modified-id="Exercise-2:-SGDClassifier-and-SGDRegresor-5">Exercise 2: <code>SGDClassifier</code> and <code>SGDRegresor</code></a></span></li><li><span><a href="#Exercise-3:-Neural-Networks-&quot;By-Hand&quot;" data-toc-modified-id="Exercise-3:-Neural-Networks-&quot;By-Hand&quot;-6">Exercise 3: Neural Networks "By Hand"</a></span></li><li><span><a href="#Exercise-4:-Predicting-Fashion" data-toc-modified-id="Exercise-4:-Predicting-Fashion-7">Exercise 4: Predicting Fashion</a></span></li><li><span><a href="#(Optional)-Exercise-5:-Implementing-Adam-Optimization-From-Scratch" data-toc-modified-id="(Optional)-Exercise-5:-Implementing-Adam-Optimization-From-Scratch-8">(Optional) Exercise 5: Implementing Adam Optimization From Scratch</a></span></li><li><span><a href="#(Optional)-Exercise-6:-Gif-or-Jiff" data-toc-modified-id="(Optional)-Exercise-6:-Gif-or-Jiff-9">(Optional) Exercise 6: Gif or Jiff</a></span></li><li><span><a href="#Submit-to-Canvas-and-GitHub" data-toc-modified-id="Submit-to-Canvas-and-GitHub-10">Submit to Canvas and GitHub</a></span></li></ul></div>

## Instructions
<hr>

rubric={mechanics:3}

**Link to your GitHub repository:**

You will receive marks for correctly submitting this assignment. To submit this assignment you should:

1. Push your assignment to your GitHub repository!
2. Provide a link to your repository in the space provided above.
2. Upload a HTML render of your assignment to Canvas. The last cell of this notebook will help you do that.
3. Be sure to follow the [General Lab Instructions](https://ubc-mds.github.io/resources_pages/general_lab_instructions/). You can view a description of the different rubrics used for grading in MDS [here](https://github.com/UBC-MDS/public/tree/master/rubric).

Here's a break down of the required and optional exercises in this lab:

|         | Number of Exercises | Points |
|:-------:|:-------------------:|:------:|
| Required| 13 | 35 |
| Optional| 2  | 2 |

## Imports
<hr>

In [ ]:
import numpy as np
import pandas as pd
import time
import torch
from torch import nn
from torchvision import datasets, transforms
from sklearn.datasets import load_boston
from sklearn.preprocessing import MinMaxScaler
from sklearn.linear_model import LinearRegression, LogisticRegression, SGDClassifier
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from canvasutils.submit import submit, convert_notebook
import matplotlib.pyplot as plt
plt.style.use('ggplot')
plt.rcParams.update({'font.size': 16, 'figure.figsize': (5,5), 'axes.grid': False})

## Exercise 1: Stochastic Gradient Descent
<hr>

Below is a super-streamlined Python function that performs gradient descent, it's much like the code you saw in Lectures and in Lab 1. I just made a few small changes:

- Removed the docstring and `print` statements for brevity.
- The algorithm stops after `n_iters`.

I also include functions to calculate the MSE and gradient of MSE for a linear model $\hat{y}=w^Tx$.

In [ ]:
def gradient_descent(f, f_grad, w0, X, y, n_iters=1000, α=0.001):
    w = w0
    for _ in range(n_iters):
        w = w - α * f_grad(w, X, y)
    return w


def mse(w, X, y):
    return np.mean((X @ w - y) ** 2)


def mse_grad(w, X, y):
    return X.T @ (X @ w) - X.T @ y

The code works fine, although it is quite slow as per usual for vanilla gradient descent. For example, here we do ordinary least squares linear regression on the boston house-prices dataset from sklearn:

In [ ]:
X, y = load_boston(return_X_y=True)
X = MinMaxScaler().fit_transform(X)

In [ ]:
start = time.time()
w0 = np.zeros(X.shape[1])
w_gd = gradient_descent(mse, mse_grad, w0, X, y, n_iters=10**5)
print(f"Fitting time = {time.time() - start:.4f}s")
print("Weights:")
w_gd

Compared to sklearn's `LinearRegression()`:

In [ ]:
start = time.time()
lr = LinearRegression(fit_intercept=False).fit(X, y)
print(f"Fitting time = {time.time() - start:.4f}s")
print("Weights:")
lr.coef_

As we can see, the coefficients obtained from gradient descent are very similar to those obtained by sklearn's `LinearRegression()` (although sklearn is much faster). All is well so far.

### 1.1
rubric={accuracy:5}

In this exercise your task is to implement a function `stochastic_gradient_descent`, that performs SGD, _using_ the `gradient_descent` function provided above. You can have your function accept the same arguments as the `gradient_descent` function above, except:

- Change `n_iters` to `n_epochs`.
- Add an extra `batch_size` argument.
- Your implementation of SGD should follow "[Approach 1](https://pages.github.ubc.ca/MDS-2020-21/DSCI_572_sup-learn-2_students/lectures/lecture3_stochastic-gradient-descent.html#sampling-with-or-without-replacement)" from Lecture 3: shuffle the dataset and then divide it into batches. If the numer of samples is not divisible by the `batch_size`, your last batch will have less than `batch_size` samples - you can either throw this last batch away or use it (I usually choose to use it and that's the default in PyTorch, the library we'll be using to build neural networks starting next lecture). 
- You can leave `α` constant for all iterations. 

>The pedagogical goal here is to help you see how SGD relates to regular "vanilla" gradient descent. In reality it would be fine to implement SGD "from scratch" without calling a GD function.

In [ ]:
def stochastic_gradient_descent(f, f_grad, w0, X, y, n_epochs=1, α=0.001, batch_size=1):
    pass  # Your solution goes here.

stochastic_gradient_descent(mse, mse_grad, w0, X, y)  # Test your function with defaults (results probably won't be very good)

### 1.2
rubric={accuracy:3}

Show that when the batch size is set to the whole training set (i.e., `batch_size=len(X)`), you get exactly the same estimated coefficients with SGD and GD. Use the same learning rate (`α=0.001`) and number of epochs (`10 ** 5`) for both algorithms.

In [ ]:
w_gd = gradient_descent(mse, mse_grad, w0, X, y, n_iters=10**5)  # GD coefficients
print(w_gd)

w_sgd = None  # Your solution goes here.
print(w_sgd)

## Exercise 2: `SGDClassifier` and `SGDRegresor`
<hr>

In this exercise we'll explore training a classifier with SGD on the [Sentiment140 dataset](http://help.sentiment140.com/home), which contains tweets labeled with sentiment associated with a brand, product, or topic. Please start by doing the following:

1. Download the corpus from [here](http://cs.stanford.edu/people/alecmgo/trainingandtestdata.zip).
2. Unzip.
3. Copy the file `training.1600000.processed.noemoticon.csv` into the current directory.
4. Create a `.gitignore` file so that you don't accidentally commit the dataset (I've tried to do this for you but please double check it).

Once you're done the above, steps, run the starter code below:

In [ ]:
# Data loading and preprocessing
df = pd.read_csv(
    "training.1600000.processed.noemoticon.csv",
    encoding="ISO-8859-1",
    names=["label", "id", "date", "no_query", "name", "text"],
)
df["label"] = df["label"].map({0: "neg", 4: "pos"})  # change 0's to "neg" and 4's to "pos"
df.head()

Now we split the data:

In [ ]:
X, y = df["text"], df["label"]
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.25, random_state=2021)

And then we encode it using `CountVectorizer`, which may take a minute or so:

In [ ]:
vec = CountVectorizer(stop_words='english')
X_train = vec.fit_transform(X_train) 
X_test = vec.transform(X_test) 

Note that our training data is rather large compared to datasets we've explored in the past:

In [ ]:
X_train.shape

Luckily, `CountVectorizer()` returns us a sparse matrix:

In [ ]:
type(X_train)

Recall that a sparse matrix is a more efficient representation of a matrix that contains many 0's. What percentage of our array is non-zero?

In [ ]:
print(f"{X_train.nnz / np.prod(X_train.shape) * 100:.5f}%")

So few non-zero values - lucky we have a sparse matrix! Anyway, let's train a classifier (this may take a while!):

In [ ]:
lr = LogisticRegression()

In [ ]:
t = time.time()
lr.fit(X_train, y_train)
print(f"Training took {time.time() - t:.1f} seconds")

In [ ]:
print(f"Train score: {lr.score(X_train, y_train):.2f}")
print(f"Test score: {lr.score(X_test, y_test):.2f}")

### 2.1
rubric={accuracy:3}

In sklearn, there is a classifier called `linear_model.SGDClassifier()` - [see the docs](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html) (there's also a `SGDRegressor` but we won't look at that here). As the name suggests, this model can train linear classifiers with SGD - in the true sense of the algorithm, 1 sample per iteration (i.e., batch size of 1).

Train a logistic regression model on the same dataset above using `SGDClassifier`. Compare the training time of your `SGDClassifier` to `LogisticRegression()`. You'll need to specify the correct `loss` argument in `SGDClassifier()` to train a logistic regression model. [Read the docstring](https://scikit-learn.org/stable/modules/generated/sklearn.linear_model.SGDClassifier.html) to find the appropriate `loss`.

> The pedagogical goal here is to demonstrate how using just one sample per iteration in SGD can significantly speed up training, and accuracy-wise, is not a terrible idea!

In [ ]:
# Your solution goes here.

### 2.2
rubric={reasoning:2}

Discuss the training and test accuracies of your `SGDClassifier()` and `LogisticRegression()` models. Is there any difference between the two models?

In [ ]:
# Your solution goes here.

### 2.3
rubric={reasoning:3}

One possible explanation for `SGDClassifier`'s speed is that it's just doing fewer iterations (epochs) before converging. `SGDClassifier` and `LogisticRegression` have an `n_iter_` attribute which you can check after fitting (in these sklearn models, `n_iter_` is equivalent to "number of epochs").
1. Compare these values and discuss them in the context of the above hypothesis.
2. Then, using the `max_iter` parameter, do a "fair" experiment where `SGDClassifier` and `LogisticRegression` do the same number of passes through the dataset, and comment on the results.

>To be completely "fair" we should also make sure that the tolerance and regularization strength in both models is the same (by default they are not) - but you don't need to worry about that here. Just focus on the number of iterations/epochs.

Your solution goes here.



## Exercise 3: Neural Networks "By Hand"
<hr>

### 3.1
rubric={accuracy:4}

The neural networks we've seen in [Lecture 4](https://pages.github.ubc.ca/MDS-2020-21/DSCI_572_sup-learn-2_students/lectures/lecture4_pytorch-neural-networks-pt1.html) are just recursive functions where each layer is made up of the previous layer's output, multiplied by some weights, with some biases added, and passed through an activation function (we call these networks "fully-connected feed-forward networks", but more on that next week). We don't usually include the input layer when counting up the layers in a network. So a 2 layer network (`L=2`) really means: 1 input layer, 1 hidden layer, 1 output layer. A 3 layer network (`L=3`) really means: 1 input layer, 2 hidden layers, 1 output layer.

Mathematically, our networks have the form:

$$ x^{(l+1)} = h\left( W^{(l)} x^{(l)} + b^{(l)}\right) $$

where:
- $W^{(l)}$ is a matrix of weights.
- $b^{(l)}$ is a vector of biases. 
- $x^{(l)}$ is the output of layer $l$:
    - $x^{(0)}$ are the inputs
    - $x^{(1)}$ are the outputs of the first hidden layers, i.e., $x^{(1)} = h\left( W^{(0)} x^{(0)} + b^{(0)}\right)$
    - etc.
    - $x^{(L)} = \hat{y}$
        - Classification: $\hat{y} = h\left( W^{(L-1)} x^{(L-1)} + b^{(L-1)}\right)$
        - Regression: $\hat{y} = W^{(L-1)} x^{(L-1)} + b^{(L-1)}$ (no activation!)

Suppose that we use a neural network with one hidden layer with a **ReLU activation** for a **regression** problem. After training, we obtain the following parameters:

$$\begin{align}W^{(0)} &= \begin{bmatrix}-2 & 2 & -1\\-1 & -2 & 0\end{bmatrix},  &b^{(0)}&=\begin{bmatrix}2 \\ 0\end{bmatrix} \\ W^{(1)} &= \begin{bmatrix}3 & 1\end{bmatrix},  &b^{(1)}&=-10\end{align}$$

For a training example with features $x = \begin{bmatrix}3 \\-2 \\ 2\end{bmatrix}$ what are the values in this network of $x^{(1)}$ and $\hat{y}$? Show your work using code cells or LaTeX.

In [ ]:
# Your answer goes here.

### 3.2
rubric={reasoning:4}

Draw this neural network using a circle/arrow diagram (similar to [the ones I drew in Lecture 4](https://pages.github.ubc.ca/MDS-2020-21/DSCI_572_sup-learn-2_students/lectures/lecture4_pytorch-neural-networks-pt1.html#neural-network-basics)). Label the diagram with the weight/bias values given above. If you want to draw this diagram by hand, that is fine: you can take a photo of the drawing and put it in here. If you are doing so, make sure you upload the image to your repo!

Your drawing goes here.



## Exercise 4: Predicting Fashion
<hr>

In this Exercise I'm going to get you to train a neural network using the Fashion-MNIST dataset. Fashion-MNIST is a set of 28 x 28 pixel greyscale images of clothes. Some of you may have worked with this dataset before - it's a classic. I promise that our datasets will get more interesting than this, but this dataset is ideal for your first PyTorch exercise. Below is a sample of some of the images in the dataset - we have 10 classes in the dataset: T-shirt/tops, Trousers, Pullovers, Dresss, Coats, Sandals, Shirts, Sneakers, Bags, and Ankle Boots.


![](img/fashion-mnist.png)

The goal of this exercise is to develop a network that can correctly predict a given image of "fashion" into one of the 10 classes. This is a multi-class classification task, our model should spit out 10 probabilities for a given image - one probability for each class. Ideally the class our model predicts with maximum probability is the correct one!

The below cell will download and load in the Fashion-MNIST data for you. We'll talk more about this process next week, but briefly:
- Think of images as ndarrays of data, in the case of grayscale images like we have here, each pixel has a value between 0 and 1 indicating how "bright" that pixel is. So each image here is just a 28 x 28 ndarray with values ranging from 0 to 1! (when we get to colour images, it's exactly the same, except each pixel has 3 values, one for each of the colour channels Red, Blue, Green. If we had colour images here our array would be 28 x 28 x 3).
- `transform`: applies some transformations to the images. Here we are converting the data to tensors. We'll work with these more next week so don't worry too much about them.
- `torch.utils.data.DataLoader`: these are "data loaders". Think of them as generators. During training/testing, we can easily query them for a batch of data of size `BATCH_SIZE`. Cool!

In [ ]:
BATCH_SIZE = 64

# Define a transform to normalize the data, which usually helps with training
transform = transforms.Compose([transforms.ToTensor()])
# Download data
trainset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=True, transform=transform)
testset = datasets.FashionMNIST('~/.pytorch/F_MNIST_data/', download=True, train=False, transform=transform)
# Create data loaders (these are just generators that will give us `batch_size` samples as a time)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)
testloader = torch.utils.data.DataLoader(testset, batch_size=64, shuffle=True)
# Class labels
class_labels = ['T-shirt/top', 'Trouser', 'Pullover', 'Dress', 'Coat',
                'Sandal', 'Shirt', 'Sneaker', 'Bag', 'Ankle Boot']

Let's plot a random image (run this cell as many times as you like to see different images):

In [ ]:
image, label = next(iter(trainloader))  # Get a random batch of 64 images
i = np.random.randint(0, 64)            # Choose one image at random
plt.imshow(image[i, 0], cmap='gray')    # Plot
plt.title(class_labels[label[i]]);

### 4.1
rubric={accuracy:3}

Notice in the plot above that our image is 28 x 28 pixels. How do we feed this into a neural network? Well we can flatten it out into a vector of 784 elements (28 x 28 = 784) and create 784 input nodes! We'll do this later on - for now, all I want you to do is create a new class defining a classifier with the following architecture ([this section of Lecture 4](https://pages.github.ubc.ca/MDS-2020-21/DSCI_572_sup-learn-2_students/lectures/lecture4_pytorch-neural-networks-pt1.html#non-linear-regression-with-a-neural-network) might help here):
- linear layer that goes from `input_size` -> 256 nodes
- ReLU activation function
- linear layer that goes from 256 nodes -> 128 nodes
- ReLU activation function
- linear layer that goes from 128 nodes -> 64 nodes
- ReLU activation function
- output layer that goes from 64 nodes -> `output_size` nodes

I've given you some starter code to get you going.

>When we create our model in a later exercise we will specify `input_size=784` and `output_size=10`. The `784` is the flattened 28 x 28 image and the output size of 10 is so that we have one node for each item of clothing (remember we have 10 classes), and each node will contain the probability of that item of clothing being the one in a particular input image.

In [ ]:
class Classifier(nn.Module):
    def __init__(self, input_size, output_size):
        super().__init__()
        self.main = None  # Your code goes here
        
    def forward(self, x):
        out = self.main(x)
        return out

### 4.2
rubric={accuracy:1}

If your model has `input_size = 784` and `output_size=10`, how many parameters does your model have? (Ideally you can work this out by hand but you can use `torchsummary` like I showed in Lecture 4 if you must...)

In [ ]:
# Your answer goes here.

### 4.3
rubric={accuracy:3}

We haven't trained yet, but let's test out your network. The below function will help you plot your network's predictions for a particular image using `matplotlib`, run it:

In [ ]:
def plot_prediction(image, label, predictions):
    """Plot network predictions with matplotlib."""
    fig, (ax1, ax2) = plt.subplots(figsize=(8, 4), ncols=2)  # Plot
    ax1.imshow(image[0], cmap='gray')
    ax1.axis('off')   
    ax1.set_title(class_labels[label])
    ax2.barh(np.arange(10), predictions.data.numpy().squeeze())   
    ax2.set_title("Predictions")
    ax2.set_yticks(np.arange(10))
    ax2.set_yticklabels(class_labels)
    ax2.set_xlim(0, 1)
    plt.tight_layout();

In [ ]:
model = Classifier(input_size=784, output_size=10)
# Test on training images (run as many times as you like!)
image, label = next(iter(trainloader))        # Get a random batch of 64 images
predictions = model(image[0].view(1, -1))     # Get first image, flatten to shape (1, 784) and predict it
predictions = nn.Softmax(dim=1)(predictions)  # Coerce predictions to probabilities using Softmax()
plot_prediction(image[0], label[0], predictions)

Okay, those predictions are probably pretty bad. We need to train! Below is a training function (the same one we saw in Lecture 4). The only difference is that when I'm creating `y_hat` (my model predictions), I'm reshaping my data to be of shape `(batch_size, 784)` using `X.view(X.shape[0], -1)` so we can feed it into our network (`X.shape[0]` is the batch size, and the -1 just says "flatten remaining dimensions into a single dimension", so this turns something of shape `(64, 28, 28)` to `(64, 784)` which in English reads as "64 examples of vectors with length 784", i.e. 64 flattened images).

In [ ]:
def trainer(model, criterion, optimizer, dataloader, epochs=5, verbose=True):
    """Simple training wrapper for PyTorch network."""

    for epoch in range(epochs):
        losses = 0
        for X, y in dataloader:
            optimizer.zero_grad()       # Clear gradients w.r.t. parameters
            y_hat = model(X.view(X.shape[0], -1))  # Reshape data to (batch_size, 784) and forward pass to get output
            loss = criterion(y_hat, y)  # Calculate loss
            loss.backward()             # Getting gradients w.r.t. parameters
            optimizer.step()            # Update parameters
            losses += loss.item()       # Add loss for this batch to running total
        if verbose: print(f"epoch: {epoch + 1}, loss: {losses / len(dataloader):.4f}")

Define an appropriate `criterion` and `optimizer` to train your model with.
- We are doing multi-class classification here, what loss function do we use for this case (hint: see [this part of Lecture 4](https://pages.github.ubc.ca/MDS-2020-21/DSCI_572_sup-learn-2_students/lectures/lecture4_pytorch-neural-networks-pt1.html#multiclass-classification-optional))?
- Use any optimizer you like but I recommend Adam (by the way, optional Exercise 5 of this lab gets you to implement Adam from scratch 😉 )

I already created the dataloader `trainloader` for you at the start of this exercise. Pass all these things to `trainer()` to train your model (it may take a few minutes):

In [ ]:
# Your answer goes here.

# Uncomment and run the below once you've defined a criterion and optimizer
# trainer(model, criterion, optimizer, trainloader, epochs=5, verbose=True)

### 4.4
rubric={accuracy:1}

Test out your newly trained network on the training data:

In [ ]:
# Test model on training images (run as many times as you like!)
image, label = next(iter(trainloader))        # Get a random batch of 64 images
predictions = model(image[0].view(1, -1))     # Get first image, flatten to shape (1, 784) and predict it
predictions = nn.Softmax(dim=1)(predictions)  # Coerce predictions to probabilities using Softmax()
plot_prediction(image[0], label[0], predictions)

And test it out on the test data:

In [ ]:
# Test model on testing images (run as many times as you like!)
image, label = next(iter(testloader))        # Get a random batch of 64 images
predictions = model(image[0].view(1, -1))     # Get first image, flatten to shape (1, 784) and predict it
predictions = nn.Softmax(dim=1)(predictions)  # Coerce predictions to probabilities using Softmax()
plot_prediction(image[0], label[0], predictions)

Pretty sweet! That's all there is to it - you just created your first legitimate classifier! WELL DONE! I personally am super excited for you.

Oh right, I need to give you an exercise to answer. In this exercise we used a `BATCH_SIZE = 64`. This is a pretty common size to use, in fact the most common sizes are: 32, 64, 128, 256, and 512. In terms of optimizing a network, list one difference between using a small batch size vs large batch size?

Your answer goes here.



### 4.5

No marks and nothing to do for this question, just a bit of fun. Our network was trained on clothing images, but that doesn't mean we can't use it to predict other images (but this is probably a bad idea)! Let's see what our model thinks of a 28 x 28 image of me:

In [ ]:
image = torch.from_numpy(plt.imread("img/tomas_beuzen.png"))
predictions = model(image.view(1, -1))        # Flatten image to shape (1, 784) and predict it
predictions = nn.Softmax(dim=1)(predictions)  # Coerce predictions to probabilities using Softmax()
label = predictions.argmax(dim=1)             # Get class label from max probability
plot_prediction(image.numpy()[None, :, :], label, predictions)

## (Optional) Exercise 5: Implementing Adam Optimization From Scratch
<hr>

rubric={accuracy:1}

Adam is an optimization algorithm that we'll be using a lot for the rest of the course. [Here's the original paper](https://arxiv.org/abs/1412.6980) that proposed it. It is essentially a fancier version of SGD. Without getting too technical, Adam really adds two additional features to SGD:
1. Momentum: which uses past gradients to help improve convergence speed, reduce noise in the path to the minimum, and avoid local minima.
2. Per-parameter learning rate: a learning rate is maintained and adapted for each parameter as iterations of optimization proceed.

Pretty cool! I recommend [reading this article](https://ruder.io/optimizing-gradient-descent/index.html) or [watching this video](https://www.youtube.com/watch?v=JXQT_vxqwIs) to learn more, but Adam boils down to the following equations:

Weight updating:

$$\mathbf{w}_{t+1} = \mathbf{w}_{t} - \frac{\alpha}{\sqrt{\hat{v}_{t}} + \epsilon} \hat{m}_{t}$$

The various components required for that equation:

$$\begin{align}
\hat{m}_{t} &= \frac{m_{t}}{1 - \beta_{1}^{t}}\\
\hat{v}_{t} &= \frac{v_{t}}{1 - \beta_{2}^{t}}
\end{align}$$

$$\begin{align}
m_{t} &= \beta_{1} m_{t-1} + (1 - \beta_{1}) g_{t}\\
v_{t} &= \beta_{2} v_{t-1} + (1 - \beta_{2}) g_{t}^{2}
\end{align}$$

Where:
- $t$ is the iteration of optimization, it increments up by one each time you update $\mathbf{w}$. Note that in the equation for $\hat{m}_{t}$ and $\hat{v}_{t}$, $\beta_{1}$ and $\beta_{2}$ are raised to the power of $t$.
- $g_{t}$ is the gradient of the loss function w.r.t to the parameters $w$.
- $m_{t}$ is known as the first moment (the mean) of the gradients. Initialize as 0.
- $v_{t}$ is known as the second moment (the uncentered variance) of the gradients. Initialize as 0.
- $\alpha$ is the learning rate. 0.1 is a good start.
- $\epsilon$ is just a term to prevent division by zero. Default: $10^{-8}$.
- $\beta_{1}$ is a hyperparameter that controls the influence of past gradients on subsequent updates. Default: $0.9$.
- $\beta_{2}$ is a hyperparameter that controls the influence of past gradients on subsequent updates. Default: $0.999$.

Here's a squiggly function for you to try and find the minimum parameter for. I've hard-coded the "optimum parameter" as $w_{opt}=4$ but I want you to find this value using Adam optimization and starting at $w \neq w_{opt}$. I've provided you the function (`f()`), the MSE loss w.r.t this function (`loss()`), and the gradient of the loss (`loss_grad()`). Run the cell below:

In [ ]:
def f(w, X):
    """Squiggly function"""
    return w * np.cos(w * X)

def loss(w, X, y):
    """MSE loss."""
    return np.mean((f(w, X) - y) ** 2)

def loss_grad(w, X, y):
    """Gradient of MSE."""
    t = np.cos(w * X) - w * X * np.sin(w * X)
    return np.mean((f(w, X) - y) * t)

w_opt = 4
X = np.arange(-3, 3, 0.1)
y = f(w_opt, X)
l = [loss(w, X, y) for w in np.arange(-10, 11, 0.1)]
plt.plot(np.arange(-10, 11, 0.1), l)
plt.xlabel("w")
plt.ylabel("MSE")
plt.grid(True);

Your task here is to implement Adam from scratch. Then use it to find $w_{opt}$ for the above function. I've provided some code below that you should run when you're ready. Note:
- I've specified a default of 100 epochs. We have a *tiny* dataset here of 60 samples so this is nothing really. Feel free to add more epochs if you wish.
- You can start with the default values for the various Adam terms I give in the equations above.
- You *may* need to play around with the hyperparameter $\alpha$ to get to the minimum (I've given a default of 0.3 in the starter code below - I didn't need to change this value in my solution). You can leave $\beta_{1}$, $\beta_{2}$ as is - often we don't tune those ones and I didn't in my solution, but you can tune them if you want.
- Adam is generally used with batches like SGD so my solution has the ability to accept a `batch_size` argument, but you don't have to code up this functionality and I didn't include that argument in the starter code below. So feel free to just use all the data each iteration for simplicity like vanilla GD would do. But if you're feeling adventurous, thrown in a `batch_size` argument 😉

>The pedagogical goal here is to get you to implement Adam and play around with it to see how it can "jump over" local minima. If you get it working, it's pretty awesome!

In [ ]:
def Adam(X, y, w0, loss, loss_grad, n_epochs=100, alpha=0.3, beta1=0.9, beta2=0.999, eta=10e-8):
    # Your code goes here.
    w = None
    return w

w0 = [9]
w = Adam(X, y, w0, loss, loss_grad)
print(w)  # Should be close to 4



## (Optional) Exercise 6: Gif or Jiff
<hr>

rubric={accuracy:1}

Practically a free mark here for making it to the end. But I do have a question for you.

.gif files are animations made of a series of images. For example, this is me when someone tells me they have 80% accuracy with their ML model:

![](img/gif.gif)

My question to you - are these files pronounced "Gif" or "Jiff"?

Your answer here: 

## Submit to Canvas and GitHub
<hr>

When you are ready to submit your assignment do the following:
1. Run all cells in your notebook to make sure there are no errors by doing `Kernel -> Restart Kernel and Run All Cells...`
2. Save your notebook.
3. Convert your notebook to `.html` format using the `convert_notebook()` function below or by `File -> Export Notebook As... -> Export Notebook to HTML`
4. Run the code `submit()` below to go through an interactive submission process to Canvas.
5. Finally, push all your work to GitHub (including the rendered html file).

In [ ]:
# convert_notebook("lab2.ipynb", "html")  # save your notebook, then uncomment and run when you want to convert to html

In [ ]:
# submit(course_code=59090)  # uncomment and run when ready to submit to Canvas

![](img/mood.png)